In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

Data preparation


In [3]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # normalize RGB channels
])

# CIFAR-10 train dataset (auto-download if not present in ./data)
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

classes = ('plane','car','bird','cat','deer',
           'dog','frog','horse','ship','truck')

100%|██████████| 170498071/170498071 [00:32<00:00, 5169011.26it/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Define CNN

In [4]:

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32x32 -> 16x16
        x = self.pool(F.relu(self.conv2(x)))  # 16x16 -> 8x8
        x = x.view(-1, 64 * 8 * 8)            # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Training setup

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# -----------------------------
# 4. Training loop

In [6]:

for epoch in range(5):  # train for 5 epochs
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[Epoch {epoch+1}, Batch {i+1}] loss: {running_loss/100:.3f}")
            running_loss = 0.0

print("✅ Training finished")

[Epoch 1, Batch 100] loss: 1.842
[Epoch 1, Batch 200] loss: 1.483
[Epoch 1, Batch 300] loss: 1.384
[Epoch 1, Batch 400] loss: 1.308
[Epoch 1, Batch 500] loss: 1.253
[Epoch 1, Batch 600] loss: 1.181
[Epoch 1, Batch 700] loss: 1.144
[Epoch 2, Batch 100] loss: 1.010
[Epoch 2, Batch 200] loss: 1.002
[Epoch 2, Batch 300] loss: 0.996
[Epoch 2, Batch 400] loss: 0.956
[Epoch 2, Batch 500] loss: 0.961
[Epoch 2, Batch 600] loss: 0.927
[Epoch 2, Batch 700] loss: 0.921
[Epoch 3, Batch 100] loss: 0.843
[Epoch 3, Batch 200] loss: 0.809
[Epoch 3, Batch 300] loss: 0.800
[Epoch 3, Batch 400] loss: 0.798
[Epoch 3, Batch 500] loss: 0.784
[Epoch 3, Batch 600] loss: 0.813
[Epoch 3, Batch 700] loss: 0.787
[Epoch 4, Batch 100] loss: 0.648
[Epoch 4, Batch 200] loss: 0.701
[Epoch 4, Batch 300] loss: 0.678
[Epoch 4, Batch 400] loss: 0.692
[Epoch 4, Batch 500] loss: 0.704
[Epoch 4, Batch 600] loss: 0.695
[Epoch 4, Batch 700] loss: 0.691
[Epoch 5, Batch 100] loss: 0.534
[Epoch 5, Batch 200] loss: 0.569
[Epoch 5, 

 Evaluation

In [7]:

correct = 0
total = 0
net.eval()

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 72.14%
